# Titanic Predictive Modeling

In this notebook we will compare several different classification algorithms to determine which algorithm is best suited for the kaggle competition. We will use the following modeling packages
* glmnet
* nnet
* gbm
* glm
* randomForest
* K-nearest neighbors

We will also be performing repeated cross validation on our models as well as parameter tuning using the `caret` package

At the end we will also be experimenting with k-nearest neighbor imputation for missing values in the age column

## Libraries and Data Reading

Our data is already cleaned of missing values and useless columns and is stored in `train_edited.csv`

In [111]:
library(caret)
library(plyr)
library(dplyr)
library(nnet)
library(glmnet)
library(gbm)
library(randomForest)
library(class)
library(doMC)
registerDoMC(cores = 3)
set.seed(42)

In [112]:
train <- read.csv('datasets/train_edited.csv', stringsAsFactors = TRUE)
train$Survived <- as.factor(train$Survived)
str(train)

'data.frame':	889 obs. of  11 variables:
 $ PassengerId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 1 2 2 ...
 $ Pclass     : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : Factor w/ 889 levels "Abbing, Mr. Anthony",..: 109 191 358 277 16 558 519 628 416 580 ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age        : num  22 38 26 35 35 ...
 $ SibSp      : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : Factor w/ 680 levels "110152","110413",..: 523 596 669 49 472 275 85 395 344 132 ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Embarked   : Factor w/ 3 levels "C","Q","S": 3 1 3 3 3 2 3 3 3 1 ...


Now we remove unecessary columns from our data that provide no current use to us

In [113]:
train <- train %>% select(-PassengerId, -Name, -Ticket)
str(train)

'data.frame':	889 obs. of  8 variables:
 $ Survived: Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 1 2 2 ...
 $ Pclass  : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Sex     : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age     : num  22 38 26 35 35 ...
 $ SibSp   : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch   : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Fare    : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Embarked: Factor w/ 3 levels "C","Q","S": 3 1 3 3 3 2 3 3 3 1 ...


## Data Scaling and Splitting

In [114]:
preProcValues <- preProcess(train, method = c("center", "scale"))
train <- predict(preProcValues, train)
str(train)

'data.frame':	889 obs. of  8 variables:
 $ Survived: Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 1 2 2 ...
 $ Pclass  : num  0.825 -1.571 0.825 -1.571 0.825 ...
 $ Sex     : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age     : num  -0.59 0.644 -0.282 0.412 0.412 ...
 $ SibSp   : num  0.431 0.431 -0.475 0.431 -0.475 ...
 $ Parch   : num  -0.474 -0.474 -0.474 -0.474 -0.474 ...
 $ Fare    : num  -0.5 0.789 -0.486 0.423 -0.484 ...
 $ Embarked: Factor w/ 3 levels "C","Q","S": 3 1 3 3 3 2 3 3 3 1 ...


Here we skip over converting our factor columns to dummy variables although this is an option. Next we use createDataPartition to split our data into train and test. Here we use a 70/30 split

In [115]:
index <- createDataPartition(train$Survived, p = .70, list = FALSE)
trainSet <- train[index, ]
testSet <- train[-index, ]
str(trainSet)

'data.frame':	623 obs. of  8 variables:
 $ Survived: Factor w/ 2 levels "0","1": 1 2 2 1 1 1 1 2 2 2 ...
 $ Pclass  : num  0.825 -1.571 -1.571 0.825 0.825 ...
 $ Sex     : Factor w/ 2 levels "female","male": 2 1 1 2 2 2 2 1 1 1 ...
 $ Age     : num  -0.59016 0.64361 0.41228 0.41228 0.00352 ...
 $ SibSp   : num  0.431 0.431 0.431 -0.475 -0.475 ...
 $ Parch   : num  -0.474 -0.474 -0.474 -0.474 -0.474 ...
 $ Fare    : num  -0.5 0.789 0.423 -0.484 -0.476 ...
 $ Embarked: Factor w/ 3 levels "C","Q","S": 3 1 3 3 2 3 3 3 1 3 ...


## Feature Selection
Here we use recursive feature elimination to select the top 5 most important predictors.

In [116]:
control <- rfeControl(functions = rfFuncs, method = "repeatedcv", repeats = 3, verbose = FALSE)
titanic_pred_prof <- rfe(select(trainSet, -Survived), pull(trainSet, Survived), rfeControl = control)
titanic_pred_prof


Recursive feature selection

Outer resampling method: Cross-Validated (10 fold, repeated 3 times) 

Resampling performance over subset size:

 Variables Accuracy  Kappa AccuracySD KappaSD Selected
         4   0.8192 0.6049    0.03770 0.08757         
         7   0.8342 0.6362    0.03048 0.06748        *

The top 5 variables (out of 7):
   Sex, Pclass, Age, Fare, Embarked


These five predictors returned by caret will be used from now on in our data analysis. We will now update our `trainSet` and `testSet` data frames to only include these `Survived`

In [117]:
trainSet <- trainSet %>% select(-Parch, -Embarked)
testSet <- testSet %>% select(-Parch, -Embarked)
str(trainSet)

'data.frame':	623 obs. of  6 variables:
 $ Survived: Factor w/ 2 levels "0","1": 1 2 2 1 1 1 1 2 2 2 ...
 $ Pclass  : num  0.825 -1.571 -1.571 0.825 0.825 ...
 $ Sex     : Factor w/ 2 levels "female","male": 2 1 1 2 2 2 2 1 1 1 ...
 $ Age     : num  -0.59016 0.64361 0.41228 0.41228 0.00352 ...
 $ SibSp   : num  0.431 0.431 0.431 -0.475 -0.475 ...
 $ Fare    : num  -0.5 0.789 0.423 -0.484 -0.476 ...


## glmnet Modeling

First we will look up the different parameters that we can modify for the glmnet model. Then we build a train control stating we will be using repeated cross validation to train our data. We specify the number of different parameter options to try for each parameter with tuneLength. Finally we train the model

In [118]:
modelLookup(model = 'glmnet')

model,parameter,label,forReg,forClass,probModel
glmnet,alpha,Mixing Percentage,TRUE,TRUE,TRUE
glmnet,lambda,Regularization Parameter,TRUE,TRUE,TRUE


In [119]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_glmnet <- train(Survived ~., data = trainSet, method = 'glmnet', trControl = trainControl, tuneLength = 25)
model_glmnet$bestTune
getTrainPerf(model_glmnet)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

,alpha,lambda
466,1,0.01607879


TrainAccuracy,TrainKappa,method
0.8096413,0.5918576,glmnet


[1] "Total time elapsed in minutes since training started 0.342666666666673"


Above we can see our glmnet parameters as well as our accuracy and kappa scores. Now we will use our model to predict on the test set

In [120]:
predictions_glmnet <- predict(model_glmnet, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_glmnet, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 145  43
         1  19  59
                                          
               Accuracy : 0.7669          
                 95% CI : (0.7114, 0.8164)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 1.294e-07       
                                          
                  Kappa : 0.4841          
 Mcnemar's Test P-Value : 0.003489        
                                          
            Sensitivity : 0.8841          
            Specificity : 0.5784          
         Pos Pred Value : 0.7713          
         Neg Pred Value : 0.7564          
             Prevalence : 0.6165          
         Detection Rate : 0.5451          
   Detection Prevalence : 0.7068          
      Balanced Accuracy : 0.7313          
                                          
       'Positive' Class : 0               
                                          

## nnet Modeling

Here we will repeat the steps of glmnet modeling except we will now use the neural net package nnet

In [121]:
modelLookup(model = 'nnet')

model,parameter,label,forReg,forClass,probModel
nnet,size,#Hidden Units,TRUE,TRUE,TRUE
nnet,decay,Weight Decay,TRUE,TRUE,TRUE


In [124]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_nnet <- train(Survived ~., data = trainSet, method = 'nnet', trControl = trainControl, tuneLength = 10)
model_nnet$bestTune
getTrainPerf(model_nnet)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

# weights:  78
initial  value 521.919047 
iter  10 value 262.034850
iter  20 value 251.836160
iter  30 value 248.078092
iter  40 value 245.276956
iter  50 value 243.820814
iter  60 value 243.503463
iter  70 value 243.158848
iter  80 value 242.941994
iter  90 value 242.743149
iter 100 value 242.676261
final  value 242.676261 
stopped after 100 iterations


,size,decay
60,11,0.1


TrainAccuracy,TrainKappa,method
0.812849,0.5939004,nnet


[1] "Total time elapsed in minutes since training started 1.34325"


In [125]:
predictions_nnet <- predict(model_nnet, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_nnet, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 147  33
         1  17  69
                                          
               Accuracy : 0.812           
                 95% CI : (0.7598, 0.8571)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 4.357e-12       
                                          
                  Kappa : 0.5903          
 Mcnemar's Test P-Value : 0.03389         
                                          
            Sensitivity : 0.8963          
            Specificity : 0.6765          
         Pos Pred Value : 0.8167          
         Neg Pred Value : 0.8023          
             Prevalence : 0.6165          
         Detection Rate : 0.5526          
   Detection Prevalence : 0.6767          
      Balanced Accuracy : 0.7864          
                                          
       'Positive' Class : 0               
                                          

## gbm Modeling

In [126]:
modelLookup(model = 'gbm')

model,parameter,label,forReg,forClass,probModel
gbm,n.trees,# Boosting Iterations,TRUE,TRUE,TRUE
gbm,interaction.depth,Max Tree Depth,TRUE,TRUE,TRUE
gbm,shrinkage,Shrinkage,TRUE,TRUE,TRUE
gbm,n.minobsinnode,Min. Terminal Node Size,TRUE,TRUE,TRUE


In [128]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_gbm <- train(Survived ~., data = trainSet, method = 'gbm', trControl = trainControl, tuneLength = 10)
model_gbm$bestTune
getTrainPerf(model_gbm)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.2407             nan     0.1000    0.0392
     2        1.1769             nan     0.1000    0.0331
     3        1.1276             nan     0.1000    0.0238
     4        1.0843             nan     0.1000    0.0200
     5        1.0482             nan     0.1000    0.0175
     6        1.0168             nan     0.1000    0.0129
     7        0.9877             nan     0.1000    0.0104
     8        0.9617             nan     0.1000    0.0114
     9        0.9400             nan     0.1000    0.0097
    10        0.9200             nan     0.1000    0.0084
    20        0.8141             nan     0.1000   -0.0005
    40        0.7137             nan     0.1000   -0.0002
    60        0.6639             nan     0.1000   -0.0028
    80        0.6287             nan     0.1000   -0.0025
   100        0.5958             nan     0.1000   -0.0021



,n.trees,interaction.depth,shrinkage,n.minobsinnode
42,100,5,0.1,10


TrainAccuracy,TrainKappa,method
0.8244155,0.6195497,gbm


[1] "Total time elapsed in minutes since training started 0.449133333333324"


In [129]:
predictions_gbm <- predict(model_gbm, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_gbm, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 151  30
         1  13  72
                                          
               Accuracy : 0.8383          
                 95% CI : (0.7885, 0.8805)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 2.231e-15       
                                          
                  Kappa : 0.647           
 Mcnemar's Test P-Value : 0.01469         
                                          
            Sensitivity : 0.9207          
            Specificity : 0.7059          
         Pos Pred Value : 0.8343          
         Neg Pred Value : 0.8471          
             Prevalence : 0.6165          
         Detection Rate : 0.5677          
   Detection Prevalence : 0.6805          
      Balanced Accuracy : 0.8133          
                                          
       'Positive' Class : 0               
                                          

## glm modeling

In [130]:
modelLookup(model = 'glm')

model,parameter,label,forReg,forClass,probModel
glm,parameter,parameter,TRUE,TRUE,TRUE


In [133]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_glm <- train(Survived ~., data = trainSet, method = 'glm', trControl = trainControl)
model_glm$bestTune
getTrainPerf(model_glm)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

parameter
none


TrainAccuracy,TrainKappa,method
0.7980697,0.5700364,glm


[1] "Total time elapsed in minutes since training started 0.0192000000000007"


In [134]:
predictions_glm <- predict(model_glm, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_glm, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 142  39
         1  22  63
                                          
               Accuracy : 0.7707          
                 95% CI : (0.7154, 0.8198)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 6.178e-08       
                                          
                  Kappa : 0.4992          
 Mcnemar's Test P-Value : 0.0405          
                                          
            Sensitivity : 0.8659          
            Specificity : 0.6176          
         Pos Pred Value : 0.7845          
         Neg Pred Value : 0.7412          
             Prevalence : 0.6165          
         Detection Rate : 0.5338          
   Detection Prevalence : 0.6805          
      Balanced Accuracy : 0.7418          
                                          
       'Positive' Class : 0               
                                          

## randomForest Modeling

In [137]:
modelLookup(model = 'rf')

model,parameter,label,forReg,forClass,probModel
rf,mtry,#Randomly Selected Predictors,TRUE,TRUE,TRUE


In [138]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_rf <- train(Survived ~., data = trainSet, method = 'rf', trControl = trainControl, tuneLength = 10)
model_rf$bestTune
getTrainPerf(model_rf)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

note: only 4 unique complexity parameters in default grid. Truncating the grid to 4 .



mtry
2


TrainAccuracy,TrainKappa,method
0.8211974,0.6124393,rf


[1] "Total time elapsed in minutes since training started 0.232233333333321"


In [139]:
predictions_rf <- predict(model_rf, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_rf, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 151  32
         1  13  70
                                          
               Accuracy : 0.8308          
                 95% CI : (0.7803, 0.8738)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 2.211e-14       
                                          
                  Kappa : 0.6292          
 Mcnemar's Test P-Value : 0.00729         
                                          
            Sensitivity : 0.9207          
            Specificity : 0.6863          
         Pos Pred Value : 0.8251          
         Neg Pred Value : 0.8434          
             Prevalence : 0.6165          
         Detection Rate : 0.5677          
   Detection Prevalence : 0.6880          
      Balanced Accuracy : 0.8035          
                                          
       'Positive' Class : 0               
                                          

## knn Modeling

In [140]:
modelLookup(model = 'knn')

model,parameter,label,forReg,forClass,probModel
knn,k,#Neighbors,TRUE,TRUE,TRUE


In [143]:
startTime <- proc.time()
trainControl <- trainControl(method = 'repeatedcv', number = 5, repeats = 5)
model_knn <- train(Survived ~., data = trainSet, method = 'knn', trControl = trainControl, tuneLength = 20)
model_knn$bestTune
getTrainPerf(model_knn)
print(paste("Total time elapsed in minutes since training started", (proc.time()[3] - startTime[3]) / 60))

,k
15,33


TrainAccuracy,TrainKappa,method
0.8051535,0.5835262,knn


[1] "Total time elapsed in minutes since training started 0.058766666666664"


In [144]:
predictions_knn <- predict(model_knn, newdata = select(testSet, -Survived), type = 'raw')
confusionMatrix(predictions_knn, testSet$Survived)

Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 147  32
         1  17  70
                                          
               Accuracy : 0.8158          
                 95% CI : (0.7639, 0.8605)
    No Information Rate : 0.6165          
    P-Value [Acc > NIR] : 1.592e-12       
                                          
                  Kappa : 0.5993          
 Mcnemar's Test P-Value : 0.0455          
                                          
            Sensitivity : 0.8963          
            Specificity : 0.6863          
         Pos Pred Value : 0.8212          
         Neg Pred Value : 0.8046          
             Prevalence : 0.6165          
         Detection Rate : 0.5526          
   Detection Prevalence : 0.6729          
      Balanced Accuracy : 0.7913          
                                          
       'Positive' Class : 0               
                                          

# Discussion

Further things to look into
* Using all features
* Results that are not reproducible - parameters change every time train is run - seeding problem because we are using doMC for parallel sampling
* knn - number of neighbors changes wildly - 11 to 33 - but accuracy still ~80%
* knn imputation for age variable

# Conclusion
Overall, the models that performed > 80% were gbm and rf with 83%. nnet, knn both had accuracies of 81% when used to predict test data values. From here it is obvious that more work must be done with modeling tuning as well as data refinement (possibly look for new features to make) to increase accuracy